# Sentiment Analysis through Bi-LSTMs with Attention Layer using Tensorflow

In this notebook, I'll be looking at how to apply recurrent neural networks to the task of sentiment analysis by using Tensorflow.

The review data is crawled from STEAM. https://store.steampowered.com/.

The model structure is inspired by "Hierarchical Attention Networks for Document Classification", Zichao Yang et al. (http://www.aclweb.org/anthology/N16-1174). 

Specifically I used LSTM cell instead of GRU cell and can better deal with long text. And use pretrained word embedding vector to accelerate training stage.

In [4]:
from __future__ import print_function, division

import os
import sys
import shutil
import numpy as np
from collections import defaultdict
from random import randint
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn

# Load pretrained GloVe word2vector model

For convenience, I used pretrained GloVe word2vector model as word embedding model. 

The model can be download here: https://nlp.stanford.edu/projects/glove/. Because of lacking resourses, I chose the model glove.6B.100d.txt. The matrix will contain 400,000 word vectors, each with a dimensionality of 100.

If some word is not in the word2vector model, I'll use zero vector to represent.

In [5]:
def loadGloveModel(gloveFilePath):
     
    print("Loading Glove Word2Vector Model...")
    f = open(gloveFilePath, 'r', encoding='utf-8')

    word_to_index = dict()
    index_to_vector = []
    for idx, line in enumerate(f):
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])

        word_to_index[word] = idx
        index_to_vector.append(embedding)
    
    f.close()
    
    # deal with word out of dictionary
    _WORD_NOT_FOUND = [0.0]* len(embedding)
    _LAST_INDEX = idx + 1
    
    word_to_index = defaultdict(lambda: _LAST_INDEX, word_to_index)
    index_to_vector += [_WORD_NOT_FOUND]
    
    print("{} words loaded!".format(len(index_to_vector)))
    return word_to_index, np.array(index_to_vector)


In [6]:
word_to_index, index_to_vector = loadGloveModel("glove.6B.100d.txt")

Loading Glove Word2Vector Model...
400001 words loaded!


# Load crawled STEAM review data

Load crawled STEAM review data and split data into training set and test set.

In [7]:
# load train & test data

def load_reviews(data_file_path, max_seq_len):
    r0_reviews = []
    r1_reviews = []
    r2_reviews = []
    r3_reviews = []

    with open(data_file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            attitude = line.split('\t')[0]
            review = line.split('\t')[1]
            if len(review) > max_seq_len:
                review = review[:max_seq_len]
            if int(attitude) == 0:
                r0_reviews.append(review)
            elif int(attitude) == 1:
                r1_reviews.append(review)
            elif int(attitude) == 2:
                r2_reviews.append(review)
            else:
                r3_reviews.append(review)

    all_reviews = np.array(r0_reviews + r1_reviews + r2_reviews + r3_reviews)
    l0, l1, l2, l3 = len(r0_reviews), len(r1_reviews), len(r2_reviews), len(r3_reviews)
    first_col = np.array([1] * l0 + [0] * l1 + [0] * l2 + [0] * l3).reshape(-1, 1)
    second_col = np.array([0] * l0 + [1] * l1 + [0] * l2 + [0] * l3).reshape(-1, 1)
    third_col = np.array([0] * l0 + [0] * l1 + [1] * l2 + [0] * l3).reshape(-1, 1)
    fourth_col = np.array([0] * l0 + [0] * l1 + [0] * l2 + [1] * l3).reshape(-1, 1)
    labels = np.concatenate([first_col, second_col, third_col, fourth_col], 1)

    print("Numbers of rating 0 reviews: {}".format(len(r0_reviews)))
    print("Numbers of rating 1 reviews: {}".format(len(r1_reviews)))
    print("Numbers of rating 2 reviews: {}".format(len(r2_reviews)))
    print("Numbers of rating 3 reviews: {}".format(len(r3_reviews)))
    
    return all_reviews, labels


In [8]:
MAX_SEQUENCE_LEN = 400
data_file_path = "data/labeled_data2"

all_reviews, labels = load_reviews(data_file_path, MAX_SEQUENCE_LEN)

Numbers of rating 0 reviews: 3040
Numbers of rating 1 reviews: 987
Numbers of rating 2 reviews: 1451
Numbers of rating 3 reviews: 4522


# Split training set and test set

In [9]:
# split train & test
X_train, X_test, y_train, y_test = train_test_split(
    all_reviews, labels, test_size=0.2, random_state=42)


# Set Parameters

In [10]:
BATCH_SIZE = 50
VECTOR_DIM = 100
LSTM_UNITS = 128
ATTENTION_SIZE = 128
NUM_CLASSES = 4

# Bi-LSTM with Attention Layer

In [11]:
def RNN(data, seq_len, lstm_units=100, attention_size=100):
    
    # bi-lstm
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(lstm_units)  # lstm_units: output dimension of each cell
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(lstm_units)
    
#     lstm_cell_1 = tf.contrib.rnn.GRUCell(lstm_units)
#     lstm_cell_2 = tf.contrib.rnn.GRUCell(lstm_units)
    
    # lstm_cell_1 = tf.contrib.rnn.DropoutWrapper(cell=lstm_cell, output_keep_prob=0.75)
    lstm_output, state = bidirectional_dynamic_rnn(lstm_cell_1, lstm_cell_2,
                                                   inputs=data, sequence_length=seq_len,
                                                   dtype=tf.float32)
    # In case of bi-lstm, output is a tuple with forward and backward output
    # output size:[batch_size, max_time, cell_fw.output_size]
    
    # attention layer
    attention_input = tf.concat(lstm_output, 2)   # concat tuple on third dimension
    
    hidden_size = attention_input.shape[2].value  # should be 2 * lstm_units
    
    # Trainable parameters
    ww = tf.Variable(tf.random_normal([hidden_size, attention_size], stddev=0.1))
    bb = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    uu = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    
    # convert B*T*D -> B*T*A
    v = tf.tanh(tf.tensordot(attention_input, ww, axes=1) + bb)
    
    # convet B*T*A -> B*T
    vu = tf.tensordot(v, uu, axes=1, name='vu')
    alphas = tf.nn.softmax(vu, name='alphas')        # attention weights
    
    attention_output = tf.reduce_sum(attention_input * tf.expand_dims(alphas, -1), 1)
    
    return attention_output
    

# Build model

Initialized embedding layer with GloVe word embedding model. This can accelerate training process and get a higher result at the begining of training. Experiment result shows that set trainable be True can get better result.

Experiment shows that use bi-lstm model can get a little better result than bi-gru model.

In [12]:
# model

tf.reset_default_graph()

with tf.name_scope('load_embedding_model'):
    tf_embedding = tf.Variable(
        tf.constant(0.0, shape=index_to_vector.shape),
        trainable=True,   # ?
        name="Embedding"
    )

    embedding_ph = tf.placeholder(tf.float32, shape=index_to_vector.shape)
    embedding_init = tf_embedding.assign(embedding_ph)

# placeholders
labels_ph = tf.placeholder(tf.float32, [None, NUM_CLASSES])
batch_ph = tf.placeholder(tf.int32, [None, MAX_SEQUENCE_LEN])
seq_len_ph = tf.placeholder(tf.int32, [None])
keep_prob_ph = tf.placeholder(tf.float32)

# embedding layer
with tf.name_scope('embedding_layer'):
    embedding_output = tf.nn.embedding_lookup(params=tf_embedding, ids=batch_ph)


# bi-lstm with attention
with tf.name_scope('rnn'):
    rnn_output = RNN(embedding_output, seq_len_ph, lstm_units=LSTM_UNITS, attention_size=ATTENTION_SIZE)
    tf.summary.histogram('rnn_outputs', rnn_output)

# dropout layer
with tf.name_scope('dropout_layer'):
    dropout = tf.nn.dropout(rnn_output, keep_prob_ph)
    dropout = dropout / keep_prob_ph

# fully connected layer
with tf.name_scope('fully_connected_layer'):
    weight = tf.Variable(tf.truncated_normal([LSTM_UNITS * 2, NUM_CLASSES]))
    bias = tf.Variable(tf.constant(0.1, shape=[NUM_CLASSES]))
    pred = tf.nn.xw_plus_b(dropout, weight, bias)
    #pred = tf.squeeze(pred)
    pred = tf.nn.softmax(pred, name='pred')


Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


# Set metrics

In [13]:
# metrics
learning_rate_ph = tf.placeholder(tf.float32)

with tf.name_scope('metrics'):
    # cross entropy loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=labels_ph))
    tf.summary.scalar('entropy_loss', loss)
    # optimizer = tf.train.AdamOptimizer().minimize(loss)
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate_ph).minimize(loss)

    # accuracy
    correctPred = tf.equal(tf.argmax(pred,1), tf.argmax(labels_ph,1))
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    
merged = tf.summary.merge_all()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



# Helper Function

In [14]:
def batch_generator(X, y, batch_size):
    """Primitive batch generator 
       copy from https://github.com/ilivans/tf-rnn-attention/blob/master/utils.py
    """
    size = X.shape[0]
    X_copy = X.copy()
    y_copy = y.copy()
    indices = np.arange(size)
    np.random.shuffle(indices)
    X_copy = X_copy[indices]
    y_copy = y_copy[indices]
    i = 0
    while True:
        if i + batch_size <= size:
            yield X_copy[i:i + batch_size], y_copy[i:i + batch_size]
            i += batch_size
        else:
            i = 0
            indices = np.arange(size)
            np.random.shuffle(indices)
            X_copy = X_copy[indices]
            y_copy = y_copy[indices]
            continue
            
            
def zero_pad(X, seq_len):
    return np.array([x[:seq_len - 1] + [0] * max(seq_len - len(x), 1) for x in X])

# Training Model

training parameters

In [15]:
# training
ITERATION = 50
EARLY_STOP = 8
PLATEAU = 3
LEARNING_RATE = 1e-3
REDUCED_LEARNING_RATE = 1e-4
DELTA = 0.5
KEEP_PROB = 1
MODEL_PATH = './labeled_model/model.ckpt'

In [16]:
# batch generator
train_batch_generator = batch_generator(X_train, y_train, BATCH_SIZE)
test_batch_generator = batch_generator(X_test, y_test, BATCH_SIZE)

# summary_writer for tensorboard
if os.path.exists('labeled_logdir/test'):
    shutil.rmtree('labeled_logdir/test')
if os.path.exists('labeled_logdir/train'):
    shutil.rmtree('labeled_logdir/train')
    
train_writer = tf.summary.FileWriter('./labeled_logdir/train', accuracy.graph)
test_writer = tf.summary.FileWriter('./labeled_logdir/test', accuracy.graph)

saver = tf.train.Saver()

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    # load word2vec into tensorflow
    _ = sess.run(embedding_init, 
                 feed_dict={embedding_ph: index_to_vector})
    
    del index_to_vector
    
    # used for early stopping
    best_accuracy_test = 0
    no_increase_count = 0
    lr = LEARNING_RATE
    
    print("training begins...")
    for epoch in range(ITERATION):
        
        loss_train = 0
        loss_test = 0
        accuracy_train = 0
        accuracy_test = 0
        
        print("epoch: {}\t".format(epoch), end="")
        
        # training
        num_batches = X_train.shape[0] // BATCH_SIZE
        
        for b in tqdm(range(num_batches)):
            x_batch_raw, y_batch = next(train_batch_generator)
            seq_len = np.array([len(s.split(' ')) for s in x_batch_raw])
            x_batch_raw = np.array([s.split(' ') for s in x_batch_raw])      # string -> word array

            x_batch = [[word_to_index[w.lower()] for w in sentense] for sentense in x_batch_raw]
            x_batch = zero_pad(x_batch, MAX_SEQUENCE_LEN)

            loss_tr, acc, _, summary = sess.run([loss, accuracy, optimizer, merged],
                                                feed_dict={batch_ph: x_batch,
                                                           labels_ph: y_batch,
                                                           seq_len_ph: seq_len,
                                                           learning_rate_ph: lr,
                                                           keep_prob_ph: KEEP_PROB})
            accuracy_train += acc
            loss_train = loss_tr * DELTA + loss_train * (1 - DELTA)
            train_writer.add_summary(summary, b + num_batches * epoch)

        
        accuracy_train /= num_batches

        # testing
        num_batches = X_test.shape[0] // BATCH_SIZE
        # num_batches = 1
        
        for b in tqdm(range(num_batches)):
            x_batch_raw, y_batch = next(test_batch_generator)
            #x_batch_raw, y_batch = X_test, y_test
            
            seq_len = np.array([len(s.split(' ')) for s in x_batch_raw])
            x_batch_raw = np.array([s.split(' ') for s in x_batch_raw])      # string -> word array

            x_batch = [[word_to_index[w.lower()] for w in sentense] for sentense in x_batch_raw]
            x_batch = zero_pad(x_batch, MAX_SEQUENCE_LEN)
            
            loss_test_batch, acc, summary = sess.run([loss, accuracy, merged],
                                                     feed_dict={batch_ph: x_batch,
                                                                labels_ph: y_batch,
                                                                seq_len_ph: seq_len,
                                                                keep_prob_ph: 1.0})
            accuracy_test += acc
            loss_test += loss_test_batch
            test_writer.add_summary(summary, b + num_batches * epoch)

        accuracy_test /= num_batches
        loss_test /= num_batches

        print("loss: {:.3f}, val_loss: {:.3f}, acc: {:.3f}, val_acc: {:.3f}".format(
            loss_train, loss_test, accuracy_train, accuracy_test
        ))
        
        if best_accuracy_test < accuracy_test:
            best_accuracy_test = accuracy_test
            saver.save(sess, MODEL_PATH)
            print("save model to" + MODEL_PATH)
            no_increase_count = 0
        else:
            print("model is not improved in current epoch!")
            no_increase_count += 1
        
        if no_increase_count == EARLY_STOP:
            print("model is not improved in the last {} epoch, early stopped!".format(EARLY_STOP))
            break
            
        if no_increase_count >= PLATEAU:
            print("model is not improved in the last {} epoch, reduced learning rate!".format(no_increase_count))
            lr = REDUCED_LEARNING_RATE
#         else:
#             lr = LEARNING_RATE
        
        
    train_writer.close()
    test_writer.close()
    # saver.save(sess, MODEL_PATH)
    
    print("training ends...")


  0%|          | 0/160 [00:00<?, ?it/s]

training begins...
epoch: 0	

100%|██████████| 40/40 [00:03<00:00, 12.23it/s]


loss: 1.250, val_loss: 1.267, acc: 0.449, val_acc: 0.464


  1%|          | 1/160 [00:00<00:27,  5.68it/s]

...

loss: 0.872, val_loss: 0.943, acc: 0.930, val_acc: 0.794
model is not improved in current epoch!
model is not improved in the last 6 epoch, reduced learning rate!
epoch: 38	

  1%|          | 1/160 [00:00<00:25,  6.22it/s]

loss: 0.794, val_loss: 0.943, acc: 0.931, val_acc: 0.792
model is not improved in current epoch!
model is not improved in the last 7 epoch, reduced learning rate!
epoch: 39	

100%|██████████| 40/40 [00:02<00:00, 13.41it/s]

loss: 0.813, val_loss: 0.942, acc: 0.932, val_acc: 0.795
model is not improved in current epoch!
model is not improved in the last 8 epoch, early stopped!
training ends...


# Get Predict Result

In [18]:
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, MODEL_PATH)

    all_data, labels = X_test, y_test
    
    seq_len = np.array([len(s.split(' ')) for s in all_data])
    input_data = np.array([s.split(' ') for s in all_data])      # string -> word array
    
    input_data = [[word_to_index[w.lower()] for w in sentense] for sentense in input_data]
    input_data = zero_pad(input_data, MAX_SEQUENCE_LEN)

    result = sess.run(pred, feed_dict={batch_ph: input_data,
                                        seq_len_ph: seq_len,
                                        keep_prob_ph: 1.0})


#result = np.concatenate([result1, result2])

print(result.shape)

INFO:tensorflow:Restoring parameters from ./labeled_model/model.ckpt
(2000, 4)


In [26]:
count_wrong = 0

for i in range(len(result)):
    if np.argmax(result[i], 0) != np.argmax(labels[i], 0):
        count_wrong += 1

print("accuracy is: {}".format(1 - count_wrong / len(result)))

accuracy is: 0.8


In [31]:
output = []
for i in result:
    output.append(np.argmax(i))

with open('text_pred_res', 'w') as f:
    f.write(str(output))